tra poco eliminerò questo file

In [10]:
from instancePDSVRP import instancePDSVRP
import time
import copy
import heuristic_min_time as hmt
import utilities as u
import heuristic as h

Method that tracks the progress of the solution (used when only one instance has to be tested)

In [11]:

def SISSRs_min_time_imported_instance(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    solutions_times = []
    s_0 = h.initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = hmt.ruin_and_recreate_min_time(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if u.makespan(instance, s) < u.makespan(instance, s_curr)*(1+delta) or ((u.makespan(instance, s) == u.makespan(instance, s_curr) and u.total_completion_time(instance,s) < u.total_completion_time(instance, s_curr))):
            s_curr = hmt.local_search_min_time(instance, s, n_nearest)
            if u.makespan(instance, s_curr) < u.makespan(instance, s_best) or ((u.makespan(instance, s) == u.makespan(instance, s_best) and u.total_completion_time(instance,s) < u.total_completion_time(instance, s_best))):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_imp:
            s_curr = h.perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        delta = delta * epsilon
        iteration_counter+=1
        solutions_times.append(u.makespan(instance, s_best))

    return s_best, solutions_times

Min Time test

In [12]:
def test_min_time(heuristic, instances, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    results = []
    
    for instance_name in instances:
        instance = instancePDSVRP("instances/" + instance_name + ".txt")
        
        print("Heuristic min time") 
        start_time = time.time()
        sol, sol_times = heuristic(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb)
        end_time = time.time()               
        time_heu = u.makespan(instance, sol)
        elapsed_time_heuristic = end_time - start_time
        print("Time found with heuristic: ", time_heu)
        
        results.append({
            "instance": instance_name,
            "time_heuristic": time_heu,
            "execution_time_heuristic": elapsed_time_heuristic,
            "solution_times": sol_times,
            "iter_imp": iter_imp,
            "iter_max": iter_max
        })
    
    return results    

In [ ]:
instances = []
imported_instance = ["eil101"]

for customers_position in ["c", "r", "rc"]:
    for depot_position in ["c", "e", "r"]:
        instances.append("30-" + customers_position + "-1-" + depot_position)
        
#instances = ["small_instances/10-c-1-c"]
        
heuristic = hmt.SISSRs_min_time
heuristic_imported_instance = SISSRs_min_time_imported_instance
        
results_our_instances = test_min_time(heuristic, instances, 0.3, 4.5, 4.5, 5,1,1,2,2, 0.1, 20, 0.1, 0.999975, 10, 100, 3, 3, 9)
results_imported_instance = test_min_time(heuristic_imported_instance, instances, 0.3, 15.15, 15.15, 5,1,1,2,2, 0.1, 20, 0.1, 0.999975, 10, 100, 3, 10.1, 9)

u.save_results_to_csv(results_our_instances, "results/heuristic_min_time_our_instances_results.csv")
u.save_results_to_csv(results_imported_instance, "results/heuristic_min_time_imported_instance_results.csv")